In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
import time
from sklearn import tree
import pandas as pd
import jsonlines
from datetime import date
import csv
import openai
openai.api_key = "xxxxxxx"

import json
import random
import torch
from torch.nn import L1Loss
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy import stats

import convokit
import spacy
from convokit import Corpus, Speaker, Utterance
from convokit import download
from convokit import TextParser

# 0. Load Data Function

## 0.1. OUM data

In [3]:
# Load the data
def load_data_oum(label='after'):
    final_convs = []
    final_labels = []
    wizards_data = []
    moral_foundations = ["care", "fairness", "liberty", "loyalty", "authority", "sanctity", "none"]
    input_files = {"wizards": "wizards_dialogues.json", "final_argubot": "argubot_final_exp.json",
                   "models_dialogues": "models_dialogues.json"}
    dials_with_scores = {"wizards": {}, "final_argubot": {}, "models_dialogues": {}}

    for key in input_files:
        input_file = input_files[key]
        with open(input_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        for d in data:
            is_wiki = False
            for m in d["messages"]:
                if 'model' in m and (m['model'] == 'wikibot' or m['model'] == 'controlbot'):
                    is_wiki = True
                    break
            if is_wiki:
                continue
            yes_no = 'none'
            k = 'Did you vote for (Leave) or against (Remain) Brexit in the 2016 UK referendum?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'against (remain)':
                    yes_no = 'no'
                elif d['participant_info'][k].lower() == 'for (leave)':
                    yes_no = 'yes'
                else:
                    yes_no = 'none'

            k = 'In the referendum on whether the UK should remain a member of the EU (BREXIT), how did you vote?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'remain (against brexit)':
                    yes_no = 'no'
                elif d['participant_info'][k].lower() == 'leave (for brexit)':
                    yes_no = 'yes'
                else:
                    yes_no = 'none'
            k = 'Have you had at least one dose of an approved Covid-19 vaccine?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'yes':
                    yes_no = 'yes'
                elif d['participant_info'][k].lower() == 'no':
                    yes_no = 'no'
            k = 'Are you a vegan?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'yes':
                    yes_no = 'yes'
                elif d['participant_info'][k].lower() == 'no':
                    yes_no = 'no'

            if yes_no == 'none':
                continue

            if 'Questions' in d['participant_info']:
                for q in d['participant_info']['Questions']:
                    if "final" in input_file:
                        if label == 'oum':
                            continue
                        if d['participant_info']['Questions'][q]['after'] == -1:
                            continue
                    elif d['participant_info']['Questions'][q]['before'] == -1 or d['participant_info']['Questions'][q]['after'] == -1:
                        continue
                    if 'good reasons' in q.lower():
                        if d['topic'] != 'brexit' and 'not' in q.lower() and yes_no == 'no':
                            continue
                        if d['topic'] != 'brexit' and 'not' not in q.lower() and yes_no == 'yes':
                            continue
                        if 'leave' in q.lower() and yes_no == 'yes':
                            continue
                        if 'remain' in q.lower() and yes_no == 'no':
                            continue
                        if d["_id"] not in dials_with_scores[key]:
                            text = ''
                            dials_with_scores[key][d["_id"]] = {"topic": d["topic"], "dataset": key}
                            for message in d['messages']:
                                if message['role'] == 'admin' or 'modified_argument' not in message:
                                    continue

                                text = text + '\n\n' + '<' + message['role'] + '>' + '\n' + message['modified_argument']
                            dials_with_scores[key][d["_id"]]['text'] = text.strip()
                            final_convs.append(text.strip())

                    if 'good reasons' in q.lower():
                        if False and label == 'oum': 
                            final_labels.append(float(d['participant_info']['Questions'][q]['after']) - float(d['participant_info']['Questions'][q]['before']))
                        else:
                            final_labels.append(float(d['participant_info']['Questions'][q]['after']))
                        oum = d['participant_info']['Questions'][q]['after'] - d['participant_info']['Questions'][q]['before'] if "final" not in input_file else None
                        dials_with_scores[key][d["_id"]]["good_reasons"] = {"oum": oum, "after": d['participant_info']['Questions'][q]['after']}
                        if 'before' in d['participant_info']['Questions'][q] and d['participant_info']['Questions'][q]['before'] != -1:
                            dials_with_scores[key][d["_id"]]["good_reasons"]['before'] = d['participant_info']['Questions'][q]['before']
                        else:
                            dials_with_scores[key][d["_id"]]["good_reasons"]['before'] = None

    assert len(final_convs) == len(final_labels)
    return final_convs, final_labels



def get_utterances(text):
    # Splitting the input text into lines
    lines = text.split('\n')
    # Variable to keep the cleaned lines
    cleaned_lines = []
    # Variable to keep track of whether the next line should be added
    add_next_line = False
    for line in lines:
        # If the line is a participant tag, set the flag to add the next line
        if line.strip() == '<participant>':
            add_next_line = True
        elif line.strip() in ['<woz>', '<chatbot>']:
            add_next_line = True
        elif add_next_line:
            # If the flag is set, add this line to the cleaned list and reset the flag
            cleaned_lines.append(line)
            add_next_line = False
    # Join the cleaned lines back into a single string
    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text

### Bot/Woz's data
conversations, labels = load_data_oum(label='after')
utterances = [get_utterances(c) for c in conversations]


## 0.2. Wikitac Data

In [5]:
import json
import pandas as pd
import numpy as np
from collections import Counter

def load_data_wikitac():
    with open('../wikitactics.json') as f:
        data = json.load(f)

    conversations = []
    utterances_cleaned = []
    labels = []
    for dispute in data:
        users = list()
        conversation = ''
        utt_cleaned = ''
        for utterance in dispute['utterances']:
            username = utterance['username']
            text = utterance['text']
            conversation += f"<user_id={username}>\n{text}\n\n"
            utt_cleaned += text + '\n\n'
        conversations.append(conversation)
        utterances_cleaned.append(utt_cleaned)
        labels.append(dispute['escalation_label'])

    return conversations, utterances_cleaned, labels

conversations, utterances, labels = load_data_wikitac()

## 0.3. AFD

In [3]:
def load_data_afd():
    # Load the data from the JSON file
    with open('../afd_1000_randomised_dialogues.json', 'r') as json_file:
        data_dict = json.load(json_file)

    conversations = data_dict['conversations']
    utterances = data_dict['utterances']
    labels = data_dict['labels']

    return conversations, utterances, labels

conversations, utterances, labels = load_data_afd()

# 4. Linguistic Features from Code Review Paper - OUM

## 4.1 Generating Linguistic Features like Style and Tones and Information Content - dataset agnostic

In [4]:
prompt = """Given an UTTERANCE, please analyze it and extract the following linguistic features:

UTTERANCE: "{utterance}"

FRAZIER Score:
- Frazier score measures the cognitive load of processing a sentence based on its syntactic structure. A high Frazier score indicates a more complex syntactic structure.
- Low Frazier Score Example: "This 'if' can be done more elegantly with Min(x,y)"
- High Frazier Score Example: "Please see this warning about adding things to NavigationEntry."

YNGVE Score:
- Yngve score is another measure of syntactic complexity, complementary to Frazier score. It quantifies the cognitive load of processing a sentence based on its constituent parse tree. A high Yngve score suggests higher complexity.
- Low Yngve Score Example: "The description is a little confusing."
- High Yngve Score Example: "The only time we call one but not the other is in the destructor, when we don't need to call needsNewGenID, but setting two fields needlessly might be a low price to pay to ensure we never accidentally call one without the other."

Propositional Density:
- Propositional density is the ratio of propositions (i.e., distinct assertions or ideas) to the total number of words in the comment. It measures information content.
- Low Propositional Density Example: "In addition to what I suggested earlier about testing for the non-existence of a third file, we could also verify that the contents of the sync database files are not nonsense."
- High Propositional Density Example: "I tried patching this in locally and it doesn't compile."

Content Density:
- Content density is the ratio of open-class words (e.g., nouns, verbs, adjectives, adverbs) to closed-class words (e.g., pronouns, prepositions, conjunctions). It is another measure of information content.
- Low Content Density Example: "Slight reordering: please put system modules first, then a blank line, then local ones (PRESUBMIT)."
- High Content Density Example: "Please check that given user id is child user, not currently active user is child."

Formality:
- Formality assesses whether the language used in the comment is formal or informal.
- Low Formality Example: "But yeah, I'm just being an API astronaut*; I think that what I wrote up there is neat, but after sleeping, don't worry about it; it's too much work to go and rewrite stuff."
- High Formality Example: "Moving this elsewhere would also keep this module focused on handling the content settings / heuristics for banners, which is what it was originally intended for."

Politeness:
- Politeness measures the degree of respect and courtesy expressed in the comment.
- Low Politeness Example: "You don't actually manage the deopt table's VirtualMemory, so you shouldn't act like you do."
- High Politeness Example: "Thanks for writing this test, getting there, but I think you could do this in a more principled way."

Sentiment:
- Sentiment analysis determines the overall emotional tone of the comment.
- Negative Sentiment Example: "That's not good use of inheritance."
- Neutral Sentiment Example: "Are we planning on making use of this other places?"
- Positive Sentiment Example: "It looks slightly magical."

Uncertainty Type:
- Uncertainty type identifies the specific type of uncertainty expressed in the comment, if any.
- Epistemic Uncertainty Example: "This seems a bit fragile."
- Doxastic Uncertainty Example: "I assume we added this notification purely for testing purposes?"
- Investigative Uncertainty Example: "Did you check whether it was needed?"
- Conditional Uncertainty Example: "Another possible option, if it does not cause user confusion, would be to automatically select those projects in the Files view when the dialog closes."

Please provide the final answer concisely and directly with no reasoning steps. For each feature, use 1 to indicate the presence and 0 to indicate the absence of the specific feature or feature option, as follows:
- Low Frazier Score: [0/1]
- High Frazier Score: [0/1]
- Low Yngve Score: [0/1]
- High Yngve Score: [0/1]
- Low Propositional Density: [0/1]
- High Propositional Density: [0/1]
- Low Content Density: [0/1]
- High Content Density: [0/1]
- Low Formality: [0/1]
- High Formality: [0/1]
- Low Politeness: [0/1]
- High Politeness: [0/1]
- Negative Sentiment: [0/1]
- Neutral Sentiment: [0/1]
- Positive Sentiment: [0/1]
- Epistemic Uncertainty: [0/1]
- Doxastic Uncertainty: [0/1]
- Investigative Uncertainty: [0/1]
- Conditional Uncertainty: [0/1]
- No Uncertainty: [0/1]
  
If multiple Uncertainty Types are detected, mark all that apply as 1. If no Uncertainty is detected, mark No Uncertainty as 1 and the rest as 0."""

In [5]:
import collections
import time
import os
import os
import json
import openai
openai.api_key = "XXXXXXXXX"


def extract_and_remove_last_word(string):
    # Split the string into lines
    lines = string.splitlines(keepends=True)

    # Extract the last word from the last line
    last_line = lines[-1]
    words = last_line.split()
    last_word = words[-1]

    # Remove the last word from the last line
    lines[-1] = ' '.join(words[:-1]) + ('\n' if last_line.endswith('\n') else '')

    # Join the lines back into a string
    string_without_last_word = ''.join(lines)

    return last_word, string_without_last_word


def prompt_gpt4_errorlessly(prompt): 
    ok = False
    while not ok:  # to avoid "ServiceUnavailableError: The server is overloaded or not ready yet."
        try:
            response = openai.ChatCompletion.create(
                        model="gpt-4-1106-preview",
                        messages=[
                                  {"role": "user", "content": prompt}
                                  ],
                        max_tokens = 200,
                        temperature = 0
                        )
            ok = True

        except Exception as ex:
            print("error", ex)
            print("too much request, sleep for 5 seconds")
            time.sleep(5)

    return response["choices"][0]["message"]["content"]

conversations, utterances, labels = load_data_afd()
annotations = collections.defaultdict(list)

prior_conversation = ''

try:
    for i, (conv) in enumerate(utterances):
        print('INDEX: ', i)
        annotations_ci = []

        for utterance in conv.split('\n\n'):
            if utterance == '' or len(utterance) == ',':
                pass
            else:
                if len(conversations[i].split(utterance)) <= 1:
                    raise 'utterance not found in the corresponding conversation'
                    
                conv_hist = conversations[i].split(utterance)[0]
                try:
                    speaker_id, conv_hist_v2 = extract_and_remove_last_word(conv_hist)
                    formatted_prompt = prompt.format(utterance=speaker_id + '\n' + utterance, conv_history=conv_hist_v2)
                    response_text = prompt_gpt4_errorlessly(formatted_prompt)
                    annotations_ci.append(response_text)
                    
                except Exception as e: # happens for one case
                    print('Error: ', e)
                    pass


        if len(response_text.split()) > 150:
            print(000000000000000000000, response_text)

        annotations['data'].append(annotations_ci)

        if (i) % 100 == 0 and i!=0:
            with open(f'afd_crlf_annotations_0-{i}.jsonl', 'w') as f:  # change afd to oum/wikitac depending on the dataset
                for key, value in annotations.items():
                    json.dump({key: value}, f)
                    f.write('\n')

    with open(f'afd_crlf_annotations_0-{i}.jsonl', 'w') as f:  # change afd to oum/wikitac depending on the dataset
        for key, value in annotations.items():
            json.dump({key: value}, f)
            f.write('\n')

except Exception as e:
    print(e)
    time.sleep(1000000)

INDEX:  0



 Given an UTTERANCE, please analyze it and extract the following linguistic features:

UTTERANCE: "Case>
Speedied six weeks ago by myself; restored per request from article creator, former member who would like to make a case for band's notability under our criteria, an opportunity I promised him. "

FRAZIER Score:
- Frazier score measures the cognitive load of processing a sentence based on its syntactic structure. A high Frazier score indicates a more complex syntactic structure.
- Low Frazier Score Example: "This 'if' can be done more elegantly with Min(x,y)"
- High Frazier Score Example: "Please see this warning about adding things to NavigationEntry."

YNGVE Score:
- Yngve score is another measure of syntactic complexity, complementary to Frazier score. It quantifies the cognitive load of processing a sentence based on its constituent parse tree. A high Yngve score suggests higher complexity.
- Low Yngve Score Example: "The description is a little confusing."
- High Y

## 4.2. Post-process the LLM's generated features

In [12]:
import re
import csv
import json
import pandas as pd

def clean_response(response):
    pattern = r"- (.*?): \[(\d)\]"
    matches = re.findall(pattern, response)

    cleaned_response = {}

    for match in matches:
        feature, value = match
        cleaned_response[feature] = int(value)

    return cleaned_response

# Load the JSONL data


# OUM
data = {}
with open('oum_crlf_annotations_0-541.jsonl') as f:
    for line in f:
        json_data = json.loads(line)
        for key, value in json_data.items():
            data.setdefault(key, []).extend(value)

# Wikitac
data = {}
with open('wikitac_crlf_annotations_0-212.jsonl') as f:
    for line in f:
        json_data = json.loads(line)
        for key, value in json_data.items():
            data.setdefault(key, []).extend(value)

# AFD
data = {}
with open('afd_crlf_annotations_0-999.jsonl') as f:
    for line in f:
        json_data = json.loads(line)
        for key, value in json_data.items():
            data.setdefault(key, []).extend(value)

# Clean the responses and convert to DataFrame format
header = ['Entity', 'Conversation', 'Response',
          'Low Propositional Density', 'High Propositional Density', 'Low Content Density', 'High Content Density', 'Low Formality',
          'High Formality', 'Low Politeness', 'High Politeness', 'Negative Sentiment', 'Neutral Sentiment', 'Positive Sentiment',
          'Epistemic Uncertainty', 'Doxastic Uncertainty', 'Investigative Uncertainty', 'Conditional Uncertainty', 'No Uncertainty']

rows = []

for key, conversations in data.items():
    for conv_idx, conversation in enumerate(conversations, start=1):
        for resp_idx, response in enumerate(conversation, start=1):
            cleaned_response = clean_response(response)
            row = [key, f"Conversation_{conv_idx}", f"Response_{resp_idx}"]

            for feature in header[3:]:
                row.append(cleaned_response.get(feature, 0))

            rows.append(row)

# Create a DataFrame from the cleaned data
df = pd.DataFrame(rows, columns=header)

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Read the data from the document_content
data = df
# Get unique conversations
conversations = data['Conversation'].unique()

# Define the features (levels and labels)
features = ['Low Propositional Density', 'High Propositional Density', 'Low Content Density', 'High Content Density', 'Low Formality',
          'High Formality', 'Low Politeness', 'High Politeness', 'Negative Sentiment', 'Neutral Sentiment', 'Positive Sentiment',
          'Epistemic Uncertainty', 'Doxastic Uncertainty', 'Investigative Uncertainty', 'Conditional Uncertainty', 'No Uncertainty']

# Initialize dictionaries to store the mean and gradient values
mean_values = defaultdict(list)
grad_values = defaultdict(list)

# Iterate over each conversation
for i, conversation in enumerate(conversations):
    
    # Iterate over each feature
    for feature in features:
        # Calculate the mean for participant data
        feature_values = data[data['Conversation']==conversation][feature].tolist()
        mean_values[feature].append(np.mean(feature_values))

        # Calculate the gradient for participant data
        x = np.arange(len(feature_values))
        y = feature_values
        try:
            slope, _ = np.polyfit(x, y, 1)
        except:
            print(i)
            slope = 0
        grad_values[feature].append(slope)



df_mean = pd.DataFrame(mean_values, index=conversations).add_suffix('_mean')
df_grad = pd.DataFrame(grad_values, index=conversations).add_suffix('_grad')
result_df = pd.concat([df_mean, df_grad], axis=1)

result_df.reset_index(inplace=True)
result_df.rename(columns={'index': 'Conversation'}, inplace=True)

# Save the result as a CSV file
del result_df['Conversation']
result_df.columns = [c for c in result_df.columns]
result_df.to_csv('1000_afd_gpt4_crlf.csv', index=False)